In [1]:
!pip install -q transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import csv
import random
import os as os
from io import StringIO
import keras
from tensorflow.keras import layers
from keras.layers import Input,Dense,concatenate,Dropout,AveragePooling2D
from keras.models import Model,load_model                                                      
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import auc,precision_recall_curve,roc_curve,confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, LSTM, RNN, SimpleRNN, Bidirectional, Concatenate, GlobalMaxPool1D, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Conv2D
from keras.utils.vis_utils import plot_model
import transformers
from transformers import T5Tokenizer, T5Model, BertModel, BertForMaskedLM, BertTokenizer, pipeline, BertConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import TFBertModel, TFXLNetModel, XLNetTokenizer,XLNetConfig, TFT5EncoderModel
import re
import pickle
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
import sentencepiece

#from keras.preprocessing.sequence import pad_sequences

print('transformers version: %s' %transformers.__version__)
print('tensorflow version: %s' %tf.__version__)
print('keras version: %s' %keras.__version__)


transformers version: 4.25.1
tensorflow version: 2.9.2
keras version: 2.9.0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def add_space_to_pep(peptides):
    peptide_space = [] 
    for ele in peptides:
        temp = [[]]
        for char in ele:
            temp.append([])
            temp[-1].append(char) 
        peptide_space.append(' '.join(''.join(ele) for ele in temp))
    peptide_space = [re.sub(r"[UZOB]", "X", sequence.lstrip()) for sequence in peptide_space]

    return peptide_space

# Pathogenic

## 1.1 TRAP softmax model 

In [5]:
sdata = pd.read_csv('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/pathogenic_db.csv')
sdata['Immunogenicity'] = sdata['Immunogenicity'].replace(['Positive', 'Positive-Low', 'Positive-Intermediate', 'Positive-High', 'Negative'], [1, 1,1,1,0])
y_train  = sdata['Immunogenicity'].values

In [6]:
protT5_xl_peptides_1 = pickle.load( open( "/content/gdrive/My Drive/1_Deep_learning/0_TRAP/bert_embedded_contacts_pathogenic/protT5_xl_peptides.pkl", "rb" ) )
print(protT5_xl_peptides_1.shape)

(10097, 7, 1024)


In [7]:
bert_embed_matrix = protT5_xl_peptides_1 
def cnn1d_1_DO(bert_embed_matrix):
    
    s = keras.Input(shape=(bert_embed_matrix.shape[1],  bert_embed_matrix.shape[2]))
    emb = bert_embed_matrix.shape[2]

    mlp_input = keras.Input(shape=(2,))

    pep_conv1 = Conv1D(emb, 1, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_1')(s)
    pep_pool1 = GlobalMaxPooling1D()(pep_conv1)
    pep_conv3 = Conv1D(emb, 3, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_3')(s)
    pep_pool3 = GlobalMaxPooling1D()(pep_conv3)
    pep_conv5 = Conv1D(emb, 5, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_5')(s)
    pep_pool5 = GlobalMaxPooling1D()(pep_conv5)
    pep_conv7 = Conv1D(emb, 7, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_7')(s)
    pep_pool7 = GlobalMaxPooling1D()(pep_conv7)
    pep_cat = concatenate([pep_pool1, pep_pool3, pep_pool5, pep_pool7])

    mlp = Dense(1500, activation='relu')(mlp_input)
    merge = concatenate([pep_cat, mlp])

    dense = Dense(256, activation='relu')(merge) 
    dense = Dropout(0.6)(dense, training = True)  # training on for monte carlo dropout 
    logits = Dense(2, name='logits')(dense)
    out = Activation('softmax')(logits)
    model = Model(inputs=[s,mlp_input], outputs=[out])

    model.compile(loss='binary_crossentropy',
                  optimizer= Adam(learning_rate = 0.00005),
                  metrics='accuracy')
    
    return model

model = cnn1d_1_DO(bert_embed_matrix)
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 1024)]    0           []                               
                                                                                                  
 kernel_1 (Conv1D)              (None, 7, 1024)      1049600     ['input_1[0][0]']                
                                                                                                  
 kernel_3 (Conv1D)              (None, 7, 1024)      3146752     ['input_1[0][0]']                
                                                                                                  
 kernel_5 (Conv1D)              (None, 7, 1024)      5243904     ['input_1[0][0]']                
                                                                                              

In [8]:
from sklearn.preprocessing import label_binarize

# dnote training data
X_train = protT5_xl_peptides_1
y_train  = sdata['Immunogenicity'].values
y_train = label_binarize([0, 1], classes= y_train).T
X_train_mlp = sdata[['hydrophobicity', 'nlog2Rank']]

# train softmax model 
array = np.arange(len(X_train))
np.random.seed(1234)
train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
valid_index = [item for item in array if item not in train_index]

input1_train = X_train[train_index]
input1_valid = X_train[valid_index]
input1_mlp_train = X_train_mlp.iloc[train_index]
input1_mlp_valid = X_train_mlp.iloc[valid_index]
label_train = y_train[train_index]
label_valid = y_train[valid_index]

random_model = cnn1d_1_DO(X_train)
callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
history = random_model.fit(
    x=[input1_train, input1_mlp_train],   # feed a list into
    y=label_train,
    validation_data = ([input1_valid, input1_mlp_valid],label_valid),
    batch_size=128,
    epochs=200,
    callbacks = [callback_val, callback_train])

softmax_model = Model(random_model.input, random_model.get_layer('logits').output)


Epoch 1/200
71/71 [==============================] - 13s 54ms/step - loss: 0.6321 - accuracy: 0.6491 - val_loss: 0.6215 - val_accuracy: 0.6525
Epoch 2/200
71/71 [==============================] - 2s 28ms/step - loss: 0.5669 - accuracy: 0.7143 - val_loss: 0.6140 - val_accuracy: 0.6564
Epoch 3/200
71/71 [==============================] - 2s 28ms/step - loss: 0.5291 - accuracy: 0.7378 - val_loss: 0.6112 - val_accuracy: 0.6772
Epoch 4/200
71/71 [==============================] - 2s 28ms/step - loss: 0.4881 - accuracy: 0.7718 - val_loss: 0.6000 - val_accuracy: 0.6941
Epoch 5/200
71/71 [==============================] - 2s 28ms/step - loss: 0.4343 - accuracy: 0.8201 - val_loss: 0.6032 - val_accuracy: 0.6812
Epoch 6/200
71/71 [==============================] - 2s 28ms/step - loss: 0.3797 - accuracy: 0.8616 - val_loss: 0.6300 - val_accuracy: 0.6871
Epoch 7/200
71/71 [==============================] - 2s 28ms/step - loss: 0.3204 - accuracy: 0.8939 - val_loss: 0.6282 - val_accuracy: 0.6723
Epoch

In [9]:
#pickle. dump(random_model_score, open('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/0_Manuscript/model/pathogenic_trap_softmax_model.pkl', 'wb'))
softmax_model.save('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/0_Manuscript/model/pathogenic_trap_softmax_model')


# Self-antigen 

## 2.1 TRAP softmax model

In [10]:
# self-antigen training dataset 
os.chdir('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/')
sdata = pd.read_csv('selfantigen_data_forMODEL.csv')

sdata['Immunogenicity'] = sdata['Immunogenicity'].replace(['Positive', 'Positive-Low', 'Positive-Intermediate', 'Positive-High', 'Negative'], [1, 1,1,1,0])
y_train  = sdata['Immunogenicity'].values

print(sdata.head())

      Peptide  Immunogenicity  \
0   AAAAAIFVI               1   
1   AAAACTTMK               1   
2   AAAALVLKA               0   
3  AAADFTAKVW               0   
4   AAADIFKHY               0   

                                     MHC_Restriction  \
0                                        HLA-B*51:01   
1                                        HLA-A*03:01   
2                HLA-B*41:01|HLA-C*12:03|HLA-C*17:01   
3  HLA-A*24:02|HLA-A*25:01|HLA-B*18:01|HLA-C*12:0...   
4  HLA-A*24:02|HLA-A*25:01|HLA-B*18:01|HLA-B*41:0...   

                               Antigen_Name   data  length ContactPosition  \
0             MHC class I related protein A   IEDB       9          AAAIFV   
1  protein ALEX isoform Alex [Homo sapiens]   IEDB       9          AACTTM   
2                                       NaN  atlas       9          AALVLK   
3                                       NaN  atlas      10         ADFTAKV   
4                                       NaN  atlas       9          ADIFKH

In [11]:
# self-antigen train embedding 
protT5_xl_peptides_1_train = pickle.load( open( "/content/gdrive/My Drive/1_Deep_learning/0_TRAP/bert_embedded_contacts_selfantigen/protT5_xl_peptides.pkl", "rb" ) )


In [12]:
bert_embed_matrix = protT5_xl_peptides_1_train 
def cnn1d_1_DO(bert_embed_matrix):
    
    s = keras.Input(shape=(bert_embed_matrix.shape[1],  bert_embed_matrix.shape[2]))
    emb = bert_embed_matrix.shape[2]

    mlp_input = keras.Input(shape=(2,))

    pep_conv1 = Conv1D(emb, 1, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_1')(s)
    pep_pool1 = GlobalMaxPooling1D()(pep_conv1)
    pep_conv3 = Conv1D(emb, 3, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_3')(s)
    pep_pool3 = GlobalMaxPooling1D()(pep_conv3)
    pep_conv5 = Conv1D(emb, 5, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_5')(s)
    pep_pool5 = GlobalMaxPooling1D()(pep_conv5)
    pep_conv7 = Conv1D(emb, 7, padding='same', activation='relu', kernel_initializer='glorot_normal', name = 'kernel_7')(s)
    pep_pool7 = GlobalMaxPooling1D()(pep_conv7)
    pep_cat = concatenate([pep_pool1, pep_pool3, pep_pool5, pep_pool7])

    mlp = Dense(1500, activation='relu')(mlp_input)
    merge = concatenate([pep_cat, mlp])

    dense = Dense(256, activation='relu')(merge) 
    dense = Dropout(0.6)(dense, training = True)  # training on for monte carlo dropout 
    logits = Dense(2, name='logits')(dense)
    out = Activation('softmax')(logits)
    model = Model(inputs=[s,mlp_input], outputs=[out])

    model.compile(loss='binary_crossentropy',
                  optimizer= Adam(learning_rate = 0.00005),
                  metrics='accuracy')
    
    return model

model = cnn1d_1_DO(bert_embed_matrix)
model.summary()



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 7, 1024)]    0           []                               
                                                                                                  
 kernel_1 (Conv1D)              (None, 7, 1024)      1049600     ['input_5[0][0]']                
                                                                                                  
 kernel_3 (Conv1D)              (None, 7, 1024)      3146752     ['input_5[0][0]']                
                                                                                                  
 kernel_5 (Conv1D)              (None, 7, 1024)      5243904     ['input_5[0][0]']                
                                                                                            

In [13]:
from sklearn.preprocessing import label_binarize

# dnote training data
X_train = protT5_xl_peptides_1_train
y_train  = sdata['Immunogenicity'].values
y_train = label_binarize([0, 1], classes= y_train).T
X_train_mlp = sdata[['hydrophobicity', 'nlog2Rank']]

# train softmax model 
array = np.arange(len(X_train))
np.random.seed(1234)
train_index = np.random.choice(array,int(len(X_train)*0.9),replace=False)
valid_index = [item for item in array if item not in train_index]

input1_train = X_train[train_index]
input1_valid = X_train[valid_index]
input1_mlp_train = X_train_mlp.iloc[train_index]
input1_mlp_valid = X_train_mlp.iloc[valid_index]
label_train = y_train[train_index]
label_valid = y_train[valid_index]

random_model = cnn1d_1_DO(X_train)
callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)
history = random_model.fit(
    x=[input1_train, input1_mlp_train],   # feed a list into
    y=label_train,
    validation_data = ([input1_valid, input1_mlp_valid],label_valid),
    batch_size=128,
    epochs=200,
    callbacks = [callback_val, callback_train])

softmax_model = Model(random_model.input, random_model.get_layer('logits').output)


Epoch 1/200
89/89 [==============================] - 4s 42ms/step - loss: 0.3318 - accuracy: 0.8791 - val_loss: 0.2799 - val_accuracy: 0.8899
Epoch 2/200
89/89 [==============================] - 3s 28ms/step - loss: 0.2412 - accuracy: 0.9079 - val_loss: 0.2596 - val_accuracy: 0.9010
Epoch 3/200
89/89 [==============================] - 3s 28ms/step - loss: 0.2044 - accuracy: 0.9225 - val_loss: 0.2482 - val_accuracy: 0.9050
Epoch 4/200
89/89 [==============================] - 3s 29ms/step - loss: 0.1755 - accuracy: 0.9342 - val_loss: 0.2392 - val_accuracy: 0.9042
Epoch 5/200
89/89 [==============================] - 3s 29ms/step - loss: 0.1484 - accuracy: 0.9479 - val_loss: 0.2363 - val_accuracy: 0.9146
Epoch 6/200
89/89 [==============================] - 3s 29ms/step - loss: 0.1132 - accuracy: 0.9633 - val_loss: 0.2571 - val_accuracy: 0.9146
Epoch 7/200
89/89 [==============================] - 3s 29ms/step - loss: 0.1041 - accuracy: 0.9687 - val_loss: 0.2442 - val_accuracy: 0.9122
Epoch 

In [14]:
#pickle. dump(random_model_score, open('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/0_Manuscript/model/self_antigen_trap_softmax_model.pkl', 'wb'))
softmax_model.save('/content/gdrive/My Drive/1_Deep_learning/0_TRAP/0_Manuscript/model/self_antigen_trap_softmax_model')
